In [21]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
from itertools import product
import pandas as pd

import os, sys
sys.path.append(os.path.join(os.getcwd(), '../sympdee/sympde'))
from misc.utils import write_lines
from data.pdes import PDEs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
def listify(comb_dict):
    for key, value in comb_dict.items():
        if not isinstance(value, list):
            comb_dict[key] = [value]
    return comb_dict

def generate_combinations(dictionary):
    # Base case: if the dictionary is empty, return an empty list
    if not dictionary:
        return [""]

    # Take one key-value pair from the dictionary
    key, values = next(iter(dictionary.items()))
    remaining_dict = dictionary.copy()
    del remaining_dict[key]

    # Recursively generate combinations for the remaining dictionary
    combinations = generate_combinations(remaining_dict)

    # Generate all possible combinations with the current key-value pair
    output_lines = []
    for value in values:
        for combination in combinations:
            if value == 'True':
                value = ''
            if value is None:
                line = combination
            elif combination == "":
                line = f"--{key} {value}" 
            else:
                if isinstance(value, list):
                    value = " ".join([str(v) for v in value])
                line = f"{combination} --{key} {value}"
            output_lines.append(line)

    return output_lines

def dict_to_array(comb_dict):
    comb_dict = listify(comb_dict)
    output_lines = generate_combinations(comb_dict)
    output_lines = "\n".join(output_lines)
    return output_lines

In [23]:
eps0to1 = np.linspace(0, 1, 3)
eps_0to1s = [[0] + [np.round(eps,1) for eps in epss] for epss in product(eps0to1, eps0to1, eps0to1)]

In [24]:
eps0to1s_nogrid = [[0,0,0,0]]
eps1 = np.linspace(0, 1, 5)[1:]
eps0 = [0]*4
for i in range(1,4):
    for eps1_i in range(len(eps1)):
        eps0_i = eps0.copy()
        eps0_i[i] = eps1[eps1_i]
        eps0to1s_nogrid.append(eps0_i)

eps0to1s_nogrid

[[0, 0, 0, 0],
 [0, 0.25, 0, 0],
 [0, 0.5, 0, 0],
 [0, 0.75, 0, 0],
 [0, 1.0, 0, 0],
 [0, 0, 0.25, 0],
 [0, 0, 0.5, 0],
 [0, 0, 0.75, 0],
 [0, 0, 1.0, 0],
 [0, 0, 0, 0.25],
 [0, 0, 0, 0.5],
 [0, 0, 0, 0.75],
 [0, 0, 0, 1.0]]

In [25]:
pdes = PDEs()
pde_names = [pde for pde in pdes]
pde_names

['Pde1',
 'Pde2',
 'Pde3',
 'Pde4',
 'Pde5',
 'Pde6',
 'Pde7',
 'Pde8',
 'Pde9',
 'Pde10',
 'Pde11',
 'Pde12',
 'Pde13',
 'Pde14',
 'Pde15',
 'Pde16',
 'Pde17',
 'Pde18',
 'Pde19',
 'Pde20',
 'Pde21',
 'Pde22',
 'Pde23',
 'Pde24',
 'KdV']

In [26]:
mlp_hidden_channels = [[100]*n_channels for n_channels in range(1, 10)]
mlp_hidden_channels

[[100],
 [100, 100],
 [100, 100, 100],
 [100, 100, 100, 100],
 [100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100, 100, 100, 100]]

In [27]:
mlp_hidden_channels_flat = [[2**16, 2**13, 2**8]]

In [28]:
seeds = [1, 2, 3]

In [29]:
n_train = [1_000, 10_000]
# n_splits_valtest = 5_000
# n_splits = [[n_split_train, n_splits_valtest, n_splits_valtest] for n_split_train in n_splits_train]

In [30]:
lrs = [0.1, 0.01, 0.001, 0.0001]
net = ['TrainP']

In [31]:
batch_sizes = list(reversed([2**i for i in range(1, 10, 2)]))
batch_sizes = list([2**i for i in range(2, 5)])
batch_sizes

[4, 8, 16]

In [32]:

comb_dicts = {
    # 'aug_noaug' : {
    #     'epsilons' : [None, [0, 1., 0.1, 0.4]],
    #     'seed': [1, 2, 3, 4, 5],
    # },
    # 'train' : {
    # },
    # 'eps_0to1_nogrid' : {
    #     'epsilons' : eps0to1s_nogrid,
    #     'seed':  seeds,
    # },
    # 'lconv' : {
    #     'rot_angle' : [0, 10, 45, 90],
    #     'dim':  [4, 9, 16, 25],
    #     'digit' : [None, 0, 5],
    # },
    # 'pdes' : {
    #     'pde_name' : pde_names,
    #     'seed':  seeds,
    # },
    # 'mlp_hidden_channels' : {
    #     'mlp_hidden_channels' : mlp_hidden_channels,
    #     'pde_name' : ['Pde22', 'Pde24', 'KdV'],
    #     'seed':  seeds,
    # },
    # 'equiv_mag_none' : {
    #     'equiv' : ['mag', 'none'],
    #     'pde_name' : pde_names,
    #     'seed':  seeds,
    # }
    # 'mlp_hidden_channels_flat' : {
    #     'mlp_hidden_channels' : mlp_hidden_channels_flat,
    #     'pde_name' : pde_names,
    #     'seed':  seeds,
    # }
    # 'transform_types' : {
    #     'transform_type' :  ['space_translation', 'none'],
    #     'seed':  np.arange(1, 101),
    # }
    "sine1d" : {
        'tags' : 'sine1d',
        'batch_size' : '2',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_high' : '3',
        'y_low' : '1',
        'noise_std' : '0.5',
        'grid_size' : '1 7',
        'eps_mult' : '0. 0. 1. 0.',
        'data_dir' : '../data/sine1d',
        'net' : 'TrainP',
        'seed':  seeds,
        'n_train' :  n_train,
        'lr' : lrs,
    },
    "sine1d-predict" : {
        'tags' : 'sine1d-predict',
        'batch_size' : '2',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_high' : '3',
        'y_low' : '1',
        'noise_std' : '0.5',
        'grid_size' : '1 7',
        'eps_mult' : '0. 0. 1. 0.',
        'data_dir' : '../data/sine1d',
        'net' : ['Predict-NoneP', 'Predict-CalculatedP', 'Predict-TrainedP'],
        'seed':  seeds,
        'n_train' :  '400',
        'lr' : '0.001'
    },
    "sine2d" : {
        'tags' : 'sine2d',
        'batch_size' : '2',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_high' : '3',
        'y_low' : '1',
        'noise_std' : '0.5',
        'grid_size' : '7 7',
        'eps_mult' : '0. 0. 1. 1.',
        'data_dir' : '../data/sine2d',
        'net' : 'TrainP',
        'n_train' :  '10000',
        'lr' : '0.001',
        'seed':  seeds,
    },
    "sine2d-predict" : {
        'tags' : 'sine2d',
        'batch_size' : '2',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_high' : '3',
        'y_low' : '1',
        'noise_std' : '0.5',
        'grid_size' : '7 7',
        'eps_mult' : '0. 0. 1. 1.',
        'data_dir' : '../data/sine2d',
        'net' : ['Predict-NoneP', 'Predict-CalculatedP', 'Predict-TrainedP'],
        'n_train' :  '1000',
        'lr' : '0.001',
        'seed':  seeds,
    },
    "sine2d-rot" : {
        'tags' : 'sine2d-rot',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_low' : '1',
        'grid_size' : '7 7',
        'eps_mult' : '0. 1. 1. 1.',
        'data_dir' : '../data/sine2d',
        'net' : 'TrainP',
        'y_high' : [3, 5],
        'noise_std' : [0.1],
        'batch_size' : [2, 16],
        'n_train' :  [10000],
        'lr' : [0.1, 0.01, 0.001],
        'seed':  [1, 2, 3],
    },
    "sine2d-rot-predict" : {
        'tags' : 'sine2d-rot-predict',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_high' : '3',
        'y_low' : '1',
        'noise_std' : '0.1',
        'grid_size' : '7 7',
        'eps_mult' : '0. 1. 1. 1.',
        'data_dir' : '../data/sine2d',
        'max_epochs' : '100',
        'n_train' :  '10000',
        'lr' : lrs,
        'seed' : seeds,
        'net' : ['Predict-NoneP', 'Predict-TrainedP'],
    },
    "flower" : {
        'tags' : 'flower',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_low' : '2',
        'y_high' : '7',
        'grid_size' : '10 10',
        'eps_mult' : '0. 1. 1. 1.',
        'data_dir' : '../data/flower',
        'net' : 'TrainP',
        'noise_std' : [0.1],
        'batch_size' : [16, 2],
        'n_train' :  [10000],
        'lr' : [0.1, 0.01, 0.001],
        'seed':  [1, 2, 3],
    },
    "flower-predict" : {
        'tags' : 'sine2d-rot-predict',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_low' : '2',
        'y_high' : '7',
        'grid_size' : '10 10',
        'eps_mult' : '0. 1. 1. 1.',
        'data_dir' : '../data/flower',
        'noise_std' : '0.1',
        'max_epochs' : '100',
        'n_train' :  '10000',
        'lr' : lrs,
        'seed' : [1, 2, 3],
        'net' : ['Predict-NoneP', 'Predict-TrainedP'],
    },
    "mnist" : {
        # 'logger' : 'None',
        'tags' : 'mnist long',
        'n_test' : '1000',
        'n_val' : '1000',
        'noise_std' : '0.',
        'grid_size' : '7 7',
        'eps_mult' : '0. 0. 1. 1.',
        'data_dir' : '../data/MNIST',
        'max_epochs' : '1000',
        'net' : 'TrainP',
        'n_train' :  '10000',
        # 'batch_size' : batch_sizes,
        'earlystop' : 'True',
        # 'lr' : lrs,
        'lr' : '0.01',
        'seed':  seeds,
    },
    "mnist-predict" : {
        # 'logger' : 'None',
        'tags' : 'mnist-predict long22',
        'n_test' : '1000',
        'n_val' : '1000',
        'noise_std' : '0.0',
        'grid_size' : '7 7',
        'eps_mult' : '0. 0. 1. 1.',
        'data_dir' : '../data/MNIST',
        'max_epochs' : '100',
        # 'net' : 'Predict-NoneP',
        'n_train' :  '10000',
        'criterion' : 'ce',
        'earlystop' : 'True',
        # 'batch_size' : batch_sizes,
        'lr' : lrs,
        'seed' : seeds,
        'net' : ['Predict-NoneP', 'Predict-TrainedP'],
    },
    "mnist-noise" : {
        # 'logger' : 'None',
        'tags' : 'mnist-noise',
        'n_test' : '1000',
        'n_val' : '1000',
        'noise_std' : '0.1',
        'grid_size' : '7 7',
        'eps_mult' : '0. 1. 1. 1.',
        'data_dir' : '../data/MNIST',
        'max_epochs' : '1000',
        'net' : 'TrainP',
        'n_train' :  '10000',
        'earlystop' : 'True',
        'lr' : [0.1, 0.01, 0.001],
        'seed':  [1, 2, 3],
    },
}


array_dir = '../jobs/arrays/'
for file_name, comb_dict in comb_dicts.items():
    output_file = os.path.join(array_dir, file_name + '.txt')
    output_lines = dict_to_array(comb_dict)
    # print(f'### {file_name} ###', comb_dict, output_lines, '\n\n',sep='\n')
    write_lines(output_file, output_lines)


In [33]:
# pd.DataFrame(comb_dicts).T.to_csv('../jobs/arrays.csv')


In [34]:
df = pd.read_csv('../jobs/arrays.csv')
df = df.set_index('experiment')
df

,tags,batch_size,max_epochs,n_test,n_val,y_high,y_low,noise_std,grid_size,eps_mult,data_dir,net,seed,n_train,lr,earlystop,criterion
experiment,,,,,,,,,,,,,,,,,
sine1d,sine1d,2,50,1000,1000,3,1.0,0.5,1 7,0. 0. 1. 0.,../data/sine1d,TrainP,"[1, 2, 3]","[1000, 10000]","[0.1, 0.01, 0.001, 0.0001]",NaN,NaN
sine1d-predict,sine1d-predict,2,50,1000,1000,3,1.0,0.5,1 7,0. 0. 1. 0.,../data/sine1d,"[Predict-NoneP, Predict-CalculatedP, Predict-T...","[1, 2, 3]",400,0.001,NaN,NaN
sine2d,sine2d,2,50,1000,1000,3,1.0,0.5,7 7,0. 0. 1. 1.,../data/sine2d,TrainP,"[1, 2, 3]",10000,0.001,NaN,NaN
sine2d-predict,sine2d-predict,2,50,1000,1000,3,1.0,0.5,7 7,0. 0. 1. 1.,../data/sine2d,"[Predict-NoneP, Predict-TrainedP]","[1, 2, 3]",1000,0.001,NaN,NaN
sine2d-rot,sine2d-rot,"[2, 16]",50,1000,1000,"[3, 5]",1.0,0.1,7 7,0. 1. 1. 1.,../data/sine2d,TrainP,"[1, 2, 3]",1000,"[0.1, 0.01, 0.001]",NaN,NaN
sine2d-rot-predict,sine2d-rot-predict,NaN,100,1000,1000,3,1.0,0.1,7 7,0. 1. 1. 1.,../data/sine2d,"[Predict-NoneP, Predict-TrainedP]","[1, 2, 3]",10000,"[0.1, 0.01, 0.001, 0.0001]",NaN,NaN
flower,flower,"[16, 2]",50,1000,1000,7,2.0,0.1,10 10,0. 1. 1. 1.,../data/flower,TrainP,"[1, 2, 3]",10000,"[0.1, 0.01, 0.001]",NaN,NaN
mnist,"[mnist, long]",NaN,1000,1000,1000,NaN,NaN,0.0,7 7,0. 1. 1. 1.,../data/MNIST,TrainP,"[1, 2, 3]",10000,0.01,True,NaN
mnist-predict,"[mnist-predict, long22]",NaN,100,1000,1000,NaN,NaN,0.0,7 7,0. 1. 1. 1.,../data/MNIST,"[Predict-NoneP, Predict-TrainedP]","[1, 2, 3]",10000,"[0.1, 0.01, 0.001, 0.0001]",True,ce


In [35]:
t = df['batch_size']
t

experiment
sine1d                      2
sine1d-predict              2
sine2d                      2
sine2d-predict              2
sine2d-rot            [2, 16]
sine2d-rot-predict        NaN
flower                [16, 2]
mnist                     NaN
mnist-predict             NaN
Name: batch_size, dtype: object

In [36]:
str(t.iloc[-1]) == 'nan'

True

In [37]:
t.iloc[-1] == np.nan

False

In [38]:
def clean_val(val):
    val = str(val)
    if ',' in val: val = val[1:-1].split(', ')
    if val == 'nan': val = np.nan
    return val

t#.apply(lambda val: clean_val(val))

experiment
sine1d                      2
sine1d-predict              2
sine2d                      2
sine2d-predict              2
sine2d-rot            [2, 16]
sine2d-rot-predict        NaN
flower                [16, 2]
mnist                     NaN
mnist-predict             NaN
Name: batch_size, dtype: object

In [39]:
for tt in t:
    tt = str(tt)
    if ',' in tt:
        tt = tt[1:-1].split(', ')
    if tt == 'nan':
        tt = np.nan
    print(tt)

2
2
2
2
['2', '16']
nan
['16', '2']
nan
nan


In [40]:
t = df['net'].iloc[1]